In [1]:
import gc
import os
import glob
import gzip
import uuid
import pickle
import zipfile
import argparse
import datetime
import warnings
import traceback

import pyart
import pandas as pd
import dask
import dask.bag as db
import numpy as np
from cftime import num2pydate as num2date

In [2]:
warnings.simplefilter("ignore")

/home/548/vhl548/.local/miniconda3/envs/radar/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
def get_calib_offset(mydate):
    """
    Get calibration offset for given date.

    Parameter:
    ==========
        mydate: datetime
            Date of treatment.

    Returns:
    ========
        calib_offset: float
            Calibration offset value. Z_calib = Z_cpol + calib_offset.
    """
    calib_offset = None
    if IS_CALIB_PERIOD:
        for datest, dateed, rval in zip(CALIB_DATE_START, CALIB_DATE_END, CALIB_VALUE):
            if (mydate >= datest) & (mydate <= dateed):
                calib_offset = rval

        # If no calibration offset has been found, then looking for the closest one.
        if calib_offset is None:
            daydelta = np.array([(cd - mydate).days for cd in CALIB_DATE_START])
            pos = np.argmax(daydelta[daydelta < 0])
            calib_offset = CALIB_VALUE[pos]
    else:
        daydelta = np.array([mydate], dtype=np.datetime64)
        pos = np.argmin(np.abs(CALIB_DATE - daydelta))
        calib_offset = CALIB_VALUE[pos]

    return calib_offset


def get_zdr_offset(mydate):
    """
    Return the ZDR calibration offset for CPOL. Requires these global variables:
    - ZDR_CALIB_DATE_START
    - ZDR_CALIB_VALUE

    Parameters:
    ===========
    mydate: datetime
        Date of treatment.

    Returns:
    ========
    off: float
        Offset in dB to apply on ZDR. ZDR_CALIB = ZDR + OFFSET
    """
    # Transform into numpy type, so that we can use numpy functions
    mydate = np.array([mydate], np.datetime64)

    pos = np.argmin(np.abs(ZDR_CALIB_DATE_START - mydate))
    off = ZDR_CALIB_VALUE[pos]

    return off


In [4]:
def create_level1a(input_file):
    try:
        radar = pyart.io.read(input_file, file_field_names=True)
    except Exception:
        print(f"Problem with {input_file}.")
        traceback.print_exc()
        return None
    
    dtime = num2date(radar.time['data'][0], radar.time['units'])
    if dtime.year > 2020:  # Wrong century...
        dtime = dtime.replace(year=dtime.year - 100)

    for dbz_name in ["DBZ", "UZ", "CZ", "Refl", None]:
        if dbz_name not in radar.fields.keys():
            continue
        else:
            break

    if dbz_name is None:
        print(f"Problem with {input_file}, no reflectivity field found.")
        return None

    zdr_calib_offset = get_zdr_offset(dtime)
    refl_calib_offset = get_calib_offset(dtime)

#     radar.instrument_parameters['radar_beam_width_h']['data'] = np.array([1], dtype=np.int32)
#     radar.instrument_parameters['radar_beam_width_v']['data'] = np.array([1], dtype=np.int32)

    fv = np.float32(-9999.0)
    deflate_level = 8
    
    radar_keys = radar.fields.keys()
    
    moment_names = ['ZDR', 'WIDTH', 'VEL', 'PHIDP']
    for m in moment_names:
        if m not in radar_keys:
            print(f"Missing {m} field in {input_file} containing only {radar_keys}.")
            return None

    refl = pyart.config.get_metadata('reflectivity')
    refl['data'] = np.ma.masked_equal(radar.fields[dbz_name]['data'].copy().filled(fv).astype(np.float32), fv)
    refl['data'] += np.float32(refl_calib_offset)
    refl['_FillValue'] = fv
    refl['_DeflateLevel'] = deflate_level
    refl['_Least_significant_digit'] = 2
    radar.add_field('DBZ', refl, replace_existing=True)

    zdr = pyart.config.get_metadata('differential_reflectivity')
    zdr['data'] = np.ma.masked_equal(radar.fields['ZDR']['data'].copy().filled(fv).astype(np.float32), fv)
    zdr['data'] += np.float32(zdr_calib_offset)
    zdr['_FillValue'] = fv
    zdr['_DeflateLevel'] = deflate_level
    zdr['_Least_significant_digit'] = 2
    radar.add_field('ZDR', zdr, replace_existing=True)

    width = pyart.config.get_metadata('spectrum_width')
    width['data'] = np.ma.masked_equal(radar.fields['WIDTH']['data'].copy().filled(fv).astype(np.float32), fv)
    width['_FillValue'] = fv
    width['_DeflateLevel'] = deflate_level
    width['_Least_significant_digit'] = 2
    radar.add_field('WIDTH', width, replace_existing=True)

    phidp = pyart.config.get_metadata('differential_phase')
    phidp['data'] = np.ma.masked_equal(radar.fields['PHIDP']['data'].copy().filled(fv).astype(np.float32), fv)
    phidp['_FillValue'] = fv
    phidp['_DeflateLevel'] = deflate_level
    phidp['_Least_significant_digit'] = 2
    radar.add_field('PHIDP', phidp, replace_existing=True)

    try:
        rhohv = pyart.config.get_metadata('cross_correlation_ratio')
        rhohv['data'] = np.ma.masked_equal(radar.fields['RHOHV']['data'].copy().filled(fv).astype(np.float32), fv)
        rhohv['_FillValue'] = fv
        rhohv['_DeflateLevel'] = deflate_level
        rhohv['_Least_significant_digit'] = 4
        radar.add_field('RHOHV', rhohv, replace_existing=True)
    except KeyError:
        pass

    vel = pyart.config.get_metadata('velocity')
    vel['data'] = np.ma.masked_equal(radar.fields['VEL']['data'].copy().filled(fv).astype(np.float32), fv)
    vel['_FillValue'] = fv
    vel['_DeflateLevel'] = deflate_level
    vel['_Least_significant_digit'] = 2
    radar.add_field('VEL', vel, replace_existing=True)

    goodkeys = ['DBZ', 'ZDR', 'WIDTH', 'PHIDP', 'RHOHV', 'VEL']
    for k in list(radar.fields.keys()):
        if k not in goodkeys:
            radar.fields.pop(k)

    metadata = dict()
    metadata['Conventions'] = radar.metadata['Conventions']
    metadata['version'] = radar.metadata['version']
    metadata['title'] = "radar PPI volume from CPOL"
    metadata['creator_name'] = 'Valentin Louf'
    metadata['creator_email'] = 'valentin.louf@bom.gov.au'
    metadata['instrument_name'] = 'CPOL'
    metadata['instrument_type'] = 'radar'
    metadata['platform_is_mobile'] = 'false'
    metadata['country'] = 'Australia'
    metadata['site_name'] = 'Gunn_Pt'
    metadata['publisher_name'] = "NCI"
    metadata['publisher_url'] = "nci.gov.au"
    metadata['institution'] = 'Australian Bureau of Meteorology'
    metadata['acknowledgement'] = 'This work has been supported by the U.S. Department of Energy Atmospheric Systems Research Program through the grant DE-SC0014063. Data may be freely distributed.'
    metadata['history'] = "created by Valentin Louf on Gadi at " + datetime.datetime.utcnow().isoformat() + " using Py-ART"
    metadata['uuid'] = str(uuid.uuid4())
    metadata['processing_level'] = 'a1'
    metadata['source'] = 'UF'
    metadata['references'] = 'cf. doi:10.1175/JTECH-D-18-0007.1'

    radar.metadata = metadata

    output_dir = os.path.join(OUTDIR, str(dtime.year))
    try:
        if not os.path.exists(output_dir):
            os.mkdir(output_dir)
    except FileExistsError:
        pass

    output_dir = os.path.join(output_dir, dtime.strftime("%Y%m%d"))
    try:
        if not os.path.exists(output_dir):
            os.mkdir(output_dir)
    except FileExistsError:
        pass

    outfilename = "twp10cpolrhi.a1.{}.{}00.nc".format(dtime.strftime("%Y%m%d"), dtime.strftime("%H%M"))
    output = os.path.join(output_dir, outfilename)

    pyart.io.write_cfradial(output, radar)
    print(f'{outfilename} written.')
    return None

In [5]:
def process_zipped_file(in_zipfile):
    with zipfile.ZipFile(in_zipfile, 'r') as zipid:
        namelst = zipid.namelist()
        uffiles = [name for name in namelst if not zipid.getinfo(name).is_dir()]
        if len(uffiles) == 0:
            print(f'No file found in zipped file {in_zipfile}.')
            return None

        for name in uffiles:
            create_level1a(zipid.open(name))
    gc.collect()
    return None

In [6]:
CALIBRATION_FILE = "/home/548/vhl548/projects/RADAR_codes/cpol_calibration/data/CALIB_OFFSET_october2017.pkl.gz"
ZDR_CALIBRATION_FILE = "/home/548/vhl548/projects/RADAR_codes/cpol_calibration/data/CPOL_ZDR_calibration_offset.pkl.gz"
with gzip.GzipFile(CALIBRATION_FILE, 'r') as gzid:
    tmp_data = pickle.load(gzid)
    # Z_CALIBRATED = Z_CPOL + CALIBRATION_VALUE
    try:
        CALIB_DATE_START = tmp_data['period_start']
        CALIB_DATE_END = tmp_data['period_end']
        CALIB_VALUE = tmp_data['calibration_value']
        IS_CALIB_PERIOD = True
    except KeyError:
        CALIB_DATE = tmp_data['date']
        CALIB_VALUE = tmp_data['calibration_value']
        IS_CALIB_PERIOD = False

# Opening calibration data file.
with gzip.GzipFile(ZDR_CALIBRATION_FILE, 'r') as gzid:
    tmp_data = pickle.load(gzid)
    ZDR_CALIB_DATE_START = tmp_data['period_start']
    ZDR_CALIB_DATE_END = tmp_data['period_end']
    ZDR_CALIB_VALUE = tmp_data['calibration_value']


In [13]:
date = datetime.datetime(2010, 3, 15)

In [14]:
get_calib_offset(date)

2.9

In [50]:
radar = pyart.io.read(infile)

/home/548/vhl548/.local/miniconda3/envs/radar/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/home/548/vhl548/.local/miniconda3/envs/radar/lib/python3.8/site-packages/pyart/io/cfradial.py:112: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  metadata = dict([(k, getattr(ncobj, k)) for k in ncobj.ncattrs()])
/home/548/vhl548/.local/miniconda3/envs/radar/lib/python3.8/site-packages/pyart/io/cfradial.py:112: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  metadata = dict([(k, getattr(ncobj, k)) for k in ncobj.ncattrs()])
/home/548/vhl548/.local/miniconda3/envs/radar/lib/python3.8/site-packages/pyart/io/cfradial.py:112: DeprecationWarning: tostring() is d

In [51]:
radar.instrument_parameters.keys()

/home/548/vhl548/.local/miniconda3/envs/radar/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


dict_keys(['frequency', 'follow_mode', 'pulse_width', 'prt_mode', 'prt', 'prt_ratio', 'polarization_mode', 'nyquist_velocity', 'unambiguous_range', 'n_samples', 'radar_antenna_gain_h', 'radar_antenna_gain_v', 'radar_beam_width_h', 'radar_beam_width_v', 'radar_rx_bandwidth', 'measured_transmit_power_v', 'measured_transmit_power_h'])

/home/548/vhl548/.local/miniconda3/envs/radar/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


['frequency',
 'prt_mode',
 'prt',
 'prt_ratio',
 'polarization_mode',
 'nyquist_velocity']

In [20]:
OUTDIR = "/scratch/kl02/vhl548/cpol/cpol_level_1a/"

In [49]:
infile = "/g/data/hj10/admin/cpol_level_0/rhi/2011/g/data/hj10/cpol_level_0/rhi/2011/20110301/cfrad.20110301_000853.000_to_20110301_000910.000_CPOL_RHI.nc"

/home/548/vhl548/.local/miniconda3/envs/radar/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [22]:
create_level1a(infile)

twp10cpolrhi.a1.20110301.000800.nc written.


In [42]:
good_keys = ['time', 'range', 'azimuth', 'elevation',
    'DBZ',
 'VEL',
 'WIDTH',
 'ZDR',
 'PHIDP',
 'RHOHV',
 'sweep_number',
 'fixed_angle',
 'sweep_start_ray_index',
 'sweep_end_ray_index',
 'sweep_mode', 
 'prt_mode',
 'prt',
 'polarization_mode',
 'nyquist_velocity',
 'radar_beam_width_h',
 'radar_beam_width_v',
 'latitude',
 'longitude',
 'altitude',
 'altitude_agl',
 'time_coverage_start',
 'time_coverage_end',
 'volume_number',
 'instrument_type']


/home/548/vhl548/.local/miniconda3/envs/radar/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [18]:
import xarray as xr

In [19]:
dset = xr.open_dataset("/scratch/kl02/vhl548/cpol/cpol_level_1a/2011/20110301/twp10cpolrhi.a1.20110301.000800.nc")
dset.radar_beam_width_h.values = 1
dset.radar_beam_width_v.values = 1

keys = list(dset.variables.keys())
bad_k = set(keys) ^ set(good_keys)
outset = dset.drop_vars(bad_k)

outset.to_netcdf("/scratch/kl02/vhl548/cpol/cpol_level_1a/2011/20110301/twp10cpolrhi.a2.20110301.000800.nc")

In [55]:
radar.metadata

/home/548/vhl548/.local/miniconda3/envs/radar/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


{'Conventions': 'CF-1.6',
 'Sub_conventions': 'CF-Radial instrument_parameters radar_parameters radar_calibration',
 'version': 'CF-Radial-1.3',
 'title': 'DARWIN',
 'institution': '',
 'references': 'NOFTNAME',
 'source': 'CAWCR',
 'comment': '',
 'original_format': 'UF',
 'driver': 'RadxConvert(NCAR)',
 'created': '2017/09/16 08:42:29.101',
 'start_datetime': '2011-03-01T00:08:53Z',
 'time_coverage_start': '2011-03-01T00:08:53Z',
 'start_time': '2011-03-01 00:08:53.000',
 'end_datetime': '2011-03-01T00:09:10Z',
 'time_coverage_end': '2011-03-01T00:09:10Z',
 'end_time': '2011-03-01 00:09:10.000',
 'instrument_name': 'CPOL',
 'site_name': '_Gunn_Pt',
 'scan_name': '',
 'scan_id': 0,
 'platform_is_mobile': 'false',
 'n_gates_vary': 'false',
 'ray_times_increase': 'true',
 'history': 'Mon Sep 18 17:57:55 2017: ncrename -v .UZ,DBZ -v VR,VEL -v SW,WIDTH -v ZD,ZDR -v PH,PHIDP -v RH,RHOHV -v .CZ,Refl ./s1011/20110301/cfrad.20110301_000853.000_to_20110301_000910.000_CPOL_RHI.nc\nGunn_Pt_rapic

/home/548/vhl548/.local/miniconda3/envs/radar/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
cpol_processing.cfmetadata.